# BIBLIOTECA

In [4]:
pip install certifi


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install requests


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip-system-certs


Usage:   
  /home/codespace/.python/current/bin/python3 -m pip <command> [options]

no such option: -s
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import ssl
import requests
import certifi
from io import StringIO
import datetime
import time

# WEB SCRAPING BCB ORIGINAL

In [5]:
ssl._create_default_https_context = ssl._create_unverified_context

# URL da tabela HTML
url = "https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?$top=100&$format=text/html"

# Leitura da tabela HTML usando pandas
tabela = pd.read_html(url)[0]  # [0] para selecionar a primeira tabela da página, se houver várias.

# Agora você tem a tabela em um DataFrame pandas chamado "tabela"
# Para ver as primeiras linhas da tabela:
tabela.head()

,id,dataMovimento,dataLiquidacao,edital,tipoPublico,prazo,quantidadeOfertada,quantidadeAceita,codigoTitulo,dataVencimento,tipoOferta,ofertante,quantidadeOfertadaSegundaRodada,quantidadeAceitaSegundaRodada,cotacaoMedia,cotacaoCorte,taxaMedia,taxaCorte,financeiro
0,9fe038043290d953509e7c2800013cd,2025-02-27 00:00:00,2025-02-28 00:00:00,48,TodoMercado,3594,500000,500000,950199,2035-01-01 00:00:00,Venda,Tesouro Nacional,125000.0,NaN,778593853,778488258,149513,14954,3893
1,9fe038043290d953509e7c2800013be,2025-02-27 00:00:00,2025-02-28 00:00:00,48,TodoMercado,2133,500000,500000,950199,2031-01-01 00:00:00,Venda,Tesouro Nacional,125000.0,NaN,838775172,83869127,149713,14974,4194
2,9fe038043290d953509e7c2800013aa,2025-02-27 00:00:00,2025-02-28 00:00:00,47,TodoMercado,2498,1000000,1000000,100000,2032-01-01 00:00:00,Venda,Tesouro Nacional,250000.0,NaN,388886769,388563686,149149,14929,3889
3,9fe038043290d953509e7c280001395,2025-02-27 00:00:00,2025-02-28 00:00:00,47,TodoMercado,1403,2000000,2000000,100000,2029-01-01 00:00:00,Venda,Tesouro Nacional,500000.0,NaN,591579107,591506878,147913,14795,11832
4,9fe038043290d953509e7c280001380,2025-02-27 00:00:00,2025-02-28 00:00:00,47,TodoMercado,762,2000000,2000000,100000,2027-04-01 00:00:00,Venda,Tesouro Nacional,500000.0,NaN,753121088,753121088,14729,14729,15062


# SELIC - RESULTADOS DOS LEILÕES

In [6]:
import ssl
import pandas as pd
from datetime import datetime

# Desabilitar verificação SSL
ssl._create_default_https_context = ssl._create_unverified_context

# Definir a data de hoje
data_hoje = datetime.now().strftime('%Y-%m-%d')

# URL da tabela HTML com a data de hoje
url = f"https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?@dataMovimentoInicio='{data_hoje}'&@dataMovimentoFim='{data_hoje}'&$top=100&$format=text/html"

# Leitura da tabela HTML usando pandas
tabela = pd.read_html(url)[0]

# 1. Reorganizar as colunas
colunas_desejadas = ['edital', 'tipoOferta', 'codigoTitulo', 'ofertante', 'prazo', 'dataLiquidacao','quantidadeOfertada', 'quantidadeAceita', 'taxaCorte']
tabela = tabela[colunas_desejadas]

# 2. Criar a coluna "NomeTitulo"
tabela['NomeTitulo'] = tabela['codigoTitulo'].map({210100: 'LFT 210100', 760199: 'NTN-B 760199'})

# 3. Mover a coluna "NomeTitulo" para a posição correta
colunas = list(tabela.columns)
colunas.insert(colunas.index('codigoTitulo') + 1, colunas.pop(colunas.index('NomeTitulo')))
tabela = tabela[colunas]

# 4. Ordenar por 'edital' com ordem personalizada
ordem_edital = [37, 38, 39]
tabela['edital'] = pd.Categorical(tabela['edital'], categories=ordem_edital, ordered=True)

# 5. Ordenar por 'prazo' com ordem personalizada
ordem_prazo = sorted(tabela['prazo'].unique())
tabela['prazo'] = pd.Categorical(tabela['prazo'], categories=ordem_prazo, ordered=True)

# 6. Criar a coluna "AnoTitulo"
tabela['AnoTitulo'] = (datetime.now() + pd.to_timedelta(pd.to_numeric(tabela['prazo'], errors='coerce'), unit='D')).dt.year

# 7. Mover a coluna "AnoTitulo" para a posição correta
colunas = list(tabela.columns)
colunas.insert(colunas.index('NomeTitulo') + 1, colunas.pop(colunas.index('AnoTitulo')))
tabela = tabela[colunas]

# 8. Ordenar o DataFrame por 'edital' e 'prazo'
tabela_ordenada = tabela.sort_values(by=['edital', 'prazo'])

# 9. Formatar a coluna "taxaCorte" como porcentagem
tabela_ordenada['taxaCorte'] = tabela_ordenada['taxaCorte'] / 10000
tabela_ordenada['taxaCorte'] = tabela_ordenada['taxaCorte'].astype(str) + '%'

# 10. Formatar a coluna "quantidadeAceita"
tabela_ordenada['quantidadeAceita'] = tabela_ordenada['quantidadeAceita'].apply(lambda x: f'{x:,.0f}'.replace(',', '.'))

# 11. Formatar a coluna "quantidadeOfertada"
tabela_ordenada['quantidadeOfertada'] = tabela_ordenada['quantidadeOfertada'].apply(lambda x: f'{x:,.0f}'.replace(',', '.'))

# Visualizar as primeiras linhas da tabela:
tabela_ordenada.head()

,edital,tipoOferta,codigoTitulo,NomeTitulo,AnoTitulo,ofertante,prazo,dataLiquidacao,quantidadeOfertada,quantidadeAceita,taxaCorte
